In [1]:
# Generic
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings, gc
warnings.filterwarnings("ignore")

# Tensorflow
import tensorflow as tf

# ktrain
import ktrain
from ktrain import text

# sklearn
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow import keras
tf.__version__

'2.10.1'

In [3]:
df = pd.read_excel("Training Data For the Topic- PersonalValuesAndBeliefs .xlsx", header='infer')
# Dropping Null Values
df.dropna(inplace=True)

# Total Records
print("Total Records: ", df.shape[0])

# Inspect
df.head()

Total Records:  10441


,Topic,Sentence
0,No,collaborating with a range of collaborating wi...
1,No,"their elders , past and present , but perhaps..."
2,No,factor in our wellbeing is nature or the natu...
3,No,percent of the world 's disease is attributab...
4,No,million deaths per year globally and 1.1 bill...


In [4]:
# Data Split
target = ['Topic']
data = ['Sentence']

X = df[data]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9396, 1) (1045, 1) (9396, 1) (1045, 1)


In [5]:
# Common Parameters
max_len = 500
batch_size = 16
learning_rate = 5e-5
epochs = 5

In [6]:
#########
## With Transformer = bert-base-uncased
######
# Transformer Model
model_ = 'bert-base-uncased'
t_mod = text.Transformer(model_, maxlen=500, classes = ['No','PersonalValuesAndBeliefs'])


'''Converting split data to list [so it can processed]'''
#train
X_tr = X_train['Sentence'].tolist()
y_tr = y_train['Topic'].tolist()

#test
X_ts = X_test['Sentence'].tolist()
y_ts = y_test['Topic'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=batch_size)

preprocessing train...
language: en
train sequence lengths:
	mean : 51
	95percentile : 67
	99percentile : 118


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 51
	95percentile : 67
	99percentile : 115


Is Multi-Label? False


In [7]:
# Train Model
learner.fit_onecycle(learning_rate, epochs)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/5
588/588 [==============================] - 41414s 70s/step - loss: 0.1401 - accuracy: 0.9537 - val_loss: 0.0161 - val_accuracy: 0.9971
Epoch 2/5
588/588 [==============================] - 40808s 69s/step - loss: 0.0159 - accuracy: 0.9960 - val_loss: 0.0052 - val_accuracy: 0.9990
Epoch 3/5
588/588 [==============================] - 40327s 69s/step - loss: 0.0100 - accuracy: 0.9976 - val_loss: 0.0056 - val_accuracy: 0.9990
Epoch 4/5
588/588 [==============================] - 38369s 65s/step - loss: 0.0033 - accuracy: 0.9994 - val_loss: 0.0066 - val_accuracy: 0.9990
Epoch 5/5
588/588 [==============================] - 37956s 65s/step - loss: 8.9574e-04 - accuracy: 0.9999 - val_loss: 0.0075 - val_accuracy: 0.9990


In [8]:
# Evaluate
x = learner.validate(class_names=t_mod.get_classes())

33/33 [==============================] - 784s 24s/step
                          precision    recall  f1-score   support

                      No       1.00      1.00      1.00       961
PersonalValuesAndBeliefs       0.99      1.00      0.99        84

                accuracy                           1.00      1045
               macro avg       0.99      1.00      1.00      1045
            weighted avg       1.00      1.00      1.00      1045



In [9]:
ktrain.get_predictor(learner.model,preproc=t_mod).save('PersonalValuesAndBeliefsBinaryClassifier2')
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)
predictor.save("./ktrain/PersonalValuesAndBeliefsClassifier2")

In [10]:
# add the name of the patient stories
df1=pd.read_excel("Patient Segments as one data entry for Bert Characterisation .xlsx")

classes = ['No','PersonalValuesAndBeliefs']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)

segmentId=df1['SegmentId']
segList=list(segmentId)

startTime=df1['StartTime']
strList=list(startTime)

endTime=df1['EndTime']
endList=list(endTime)

transcript=df1['Transcript']
transList=list(transcript)

In [10]:
#To write the transcript as a paragraph to be passes to the predictor
i=0
segTranscript=[]
allSegmentsTrans=[]
startime=strList[0]
endTime=endList[0]
segTrigger=i
while i < len(segList)-2:
    #print(segList[i])
    #print()
    if segList[i]==segList[i+1]:
        print(segList[i],'  ==  ',segList[i+1])
        print()
        segTranscript.append(transList[i])
        #segTranscript.append(transList1[i+1])
        print( transList[i])
        #print( transList1[i+1])
        print()
        i+=1
    else:
        print(segList[i],'  !=  ',segList[i+1])
        #print()
        #print('start time= ',strList[i+1],'    ','end time= ',endList[i+1])

        segTranscript.append(transList[i])
        listToString=' '.join([str(item) for item in segTranscript ])
        allSegmentsTrans.append([segList[i],startime,endTime,segTranscript,listToString])
        print(listToString)
        print()
        segTranscript=[]
        
        startime=strList[i+1]
        endTime=endList[i+1]
        
        i+=1

MxmrjLQvg7E1   !=   MxmrjLQvg7E2
cancer cancer and there's there's it's spread everywhere and there's nothing we can do. I remember I was holding on to my mum screaming, don't let me die and she mum was just going I won't you're, not die and i was going. just don't let me

MxmrjLQvg7E2   !=   MxmrjLQvg7E3
 that I completely lost my vision. So everything just went black and i suffered with horrific migraines. So i got kind of i didn't got rushed into hospital, but i went for an eye test first and at my test at boots. That'S where

MxmrjLQvg7E3   !=   MxmrjLQvg7E4
 there's nothing we can do. it's spread all over. The body and she needs and my mom said staying in This hospital, to for her days. We want to take her home so a family, they brought me home and i had 24-hour care. I had

MxmrjLQvg7E4   !=   MxmrjLQvg7E5
 and now, i'm on in a it's like. I tell myself that it's like. it's i'm like. it's just it's like. chronic illness. There'S people with ms there's people that there's people th

Tyo2YsA0Y03   !=   Tyo2YsA0Y04
 And I have to put all my energy and presence in taking care of this issue . It wa in the very early stage so treatment for me didn ’ t involve any chemo didn ’ t involve any radiation . The surgery is considered a major surgery but when I think of the timeframe from diagnosis to being able to get back into my life it wa a matter of six week from having my upper right lobe removed in which the cancer wa all encapsulated in that lobe so it hadn ’ t metastasized hadn ’ t spread .

Tyo2YsA0Y04   !=   Tyo2YsA0Y05
 They were very very small so I wa able very kind of succinctly and simply have treatment and be able to be back into my life in six week . I don ’ t think I can say too much about the fact that I wa so lucky that this cancer wa found early . The care every step of the way there wasn ’ t a person that I dealt with from going to have x done to having the actual surgery to post to care in the hospital follow up of phone call when I came home .

Tyo2YsA

In [11]:
#### 
# Write the annotation result to Excel File- old videos

import xlrd
import xlsxwriter
workbook = xlsxwriter.Workbook('Health Domain-PatientStories-BinaryClassifier Bert-PersonalValuesAndBeliefs2.xlsx')
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
worksheet.write('E1', 'Topic')
row=1
col=0
transRow=0

for i in range(0,len(segList)):
    
    worksheet.write(row,col,segList[i])
    worksheet.write(row,col+1,strList[i])
    worksheet.write(row,col+2,endList[i])
    transRow=row
    #for j in i[3]:
    #    if str(j)!='nan':
    #        worksheet.write(row,col+3,j)
    #        row+=1
    #print('prediction= ',predictor.predict(i[4]))
    #print()
    worksheet.write(row,col+3,transList[i])
    worksheet.write(row,col+4,predictor.predict(transList[i]))
    row+=1
    col=0
        
        

    
workbook.close()  


1/1 [==============================] - 0s 171ms/step


1/1 [==============================] - 0s 122ms/step


1/1 [==============================] - 0s 165ms/step


1/1 [==============================] - 0s 169ms/step


1/1 [==============================] - 0s 170ms/step


1/1 [==============================] - 0s 172ms/step


1/1 [==============================] - 0s 162ms/step


1/1 [==============================] - 0s 158ms/step
